# Explore models
Try SOTA text summarization models for their performance on speech/lecture transcripts
Since Sat. Oct. 23rd, 2021


In [4]:
## Setup
import torch
# import tensorflow
from transformers import pipeline
from transformers import AutoModelWithLMHead, AutoTokenizer
from transformers import PegasusTokenizer, BigBirdPegasusForConditionalGeneration, BigBirdPegasusConfig
# import protobuf

from util import *


txt_ted = get_ted_eg()['transcript']
print(txt_ted)
ic(len(txt_ted.split()))
txt_498 = get_498_eg()
txt_498_sec = get_498_eg(section=True)
ic(len(txt_498.split()))
ic(len(txt_498_sec.split()))



ic| len(txt_ted.split()): 3170
ic| len(txt_498.split()): 10548
ic| len(txt_498_sec.split()): 3561


 Good morning. How are you? 
 (Audience) Good. 
 It's been great, hasn't it? I've been blown away by the whole thing. In fact, I'm leaving. 
 (Laughter) 
 There have been three themes running through the conference, which are relevant to what I want to talk about. One is the extraordinary evidence of human creativity in all of the presentations that we've had and in all of the people here; just the variety of it and the range of it. The second is that it's put us in a place where we have no idea what's going to happen in terms of the future. No idea how this may play out. 
 I have an interest in education. Actually, what I find is, everybody has an interest in education. Don't you? I find this very interesting. If you're at a dinner party, and you say you work in education - actually, you're not often at dinner parties, frankly. 
 (Laughter) 
 If you work in education, you're not asked. 
 (Laughter) 
 And you're never asked back, curiously. That's strange to me. But if you are, and you

3561

## BigBird
"Big Bird: Transformers for Longer Sequences".
Manzil Zaheer, Guru Guruganesh, Kumar Avinava Dubey, Joshua Ainslie, Chris Alberti, Santiago Ontanon, Philip Pham, Anirudh Ravula, Qifan Wang, Li Yang, Amr Ahmed.
*NeurIPS 2020*.



In [2]:
def bigbird(text):
    """
    `BigBird` in torch

    Looks like doesn't support summarization
    """
    summarizer = pipeline(
        'summarization',
        model='google/bigbird-roberta-base',
        tokenizer='google/bigbird-roberta-base'
    )
    summarizer(text, min_length=5, max_length=20)
# bigbird('hello world')


def bigbird_pegasus(text):
    model = BigBirdPegasusForConditionalGeneration.from_pretrained('google/bigbird-pegasus-large-arxiv')
    tokenizer = PegasusTokenizer.from_pretrained('google/bigbird-pegasus-large-arxiv')
    inputs = tokenizer([text], max_length=4096, return_tensors='pt', truncation=True)
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=256, early_stopping=True)
    return tokenizer.batch_decode(summary_ids)
"""
Hyper-parameter seems to affect the result drastically

On TED.com `Do schools kill creativity?`
['<s> we present a brief discussion of the nature of education in the era of '
'big data and big rip.<n> we start with a brief history of education in the '
'era of big rip.<n> we then turn to a brief discussion of the nature of '
'education in the era of big rip.<n> we conclude with a discussion of the '
'future of education.']

['<s> this is an elementary introduction to the notion of the distance between '
'two points on the unit circle.<n> the distance is defined as the difference '
'between the value of the unit circle and the value of the point on the unit '
'circle.<n> the definition of the distance is based on the concept of an '
'equivalence between the value of the unit circle and the value of the point '
'on the unit circle.<n> the distance is defined as the difference between the '
'value of the unit circle and the value of the point on the unit circle.<n> '
'the definition of the distance is based on the concept of an equivalence '
'between the value of the unit circle and the value of the point on the unit '
'circle. <n> [ [ section ] ] this is an elementary introduction to the notion '
'of the distance between two points on the unit circle.<n> the distance is '
'defined as the difference between the value of the unit circle and the value '
'of the point on the unit circle.<n> the definition of the distance is based '
'on the concept of an equivalence between the value of the unit circle and '
'the value of the point on the unit circle.<n> the distance is defined as the '
'difference between the value of the unit circle and the value of the point '
'on the unit circle ']
"""


ic(txt_498[:400])
# txt = get_1st_n_words(txt)
# ic(txt)
ic(bigbird_pegasus(txt_ted))
# ic(bigbird_pegasus(txt_498))




ic| txt_498[:400]: ("No, you don't need a reason. Apple TV. All right. Apple TV. Sweet don't have "
                    "to carry around a dongle. Welcome, welcome everybody. Yeah, it's um the "
                    "gloomy day. I got a bunch of great emails saying I can't make it. I feel it. "
                    "I feel it. But anyway, let's just jump right into how many people have "
                    "groups? Awesome. That's all you left. The scene. Pretty much everybody. So "
                    "we'll will tease out who mad")


KeyboardInterrupt: 

## T5



In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")


def summarize(text, max_length=150):
    input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)
    generated_ids = model.generate(input_ids=input_ids, num_beams=2, max_length=max_length,  repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)
    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
    return preds[0]


ic(summarize(txt_ted))



## Distill BART
Token size of 1024 is not large enough



In [ ]:
# summarizer = pipeline('summarization', model='sshleifer/distilbart-cnn-12-6')
summarizer = pipeline('summarization', model='facebook/bart-large-cnn')
ic(summarizer(txt_ted, max_length=300))


